#  Predição de Preços de Carros usando KNN
# 
### Este notebook implementa um modelo KNN (K-Nearest Neighbors) para prever preços de carros.

### 1. Importação das Bibliotecas


In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OrdinalEncoder


### 2. Funções Auxiliares

In [2]:
def knn_input(df, n_neighbors=5):   
    df_encoded = df.copy()
    for col in df_encoded.select_dtypes(include='object').columns:
        df_encoded[col] = df_encoded[col].astype('category').cat.codes
    knn_imputer = KNNImputer(n_neighbors=n_neighbors)
    df_imputed = pd.DataFrame(knn_imputer.fit_transform(df_encoded), columns=df_encoded.columns)
    for col in df.select_dtypes(include='object').columns:
        df_imputed[col] = df_imputed[col].round().astype(int).map(
            dict(enumerate(df[col].astype('category').cat.categories)))
    return df_imputed

def remove_outliers_iqr(df, column):   
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1   
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR    
    df_out = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df_out

### 3. Carregamento e Pré-processamento dos Dados

In [3]:
train_path = '../data/raw/train.csv'
df = pd.read_csv(train_path)
df.drop(axis=1, columns=['id'], inplace=True)

In [4]:
df_input = knn_input(df, n_neighbors=25)


cat_cols_train = df_input.select_dtypes(include=['object']).columns
cat_cols_train = cat_cols_train[cat_cols_train != 'class']
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)


df_input[cat_cols_train] = ordinal_encoder.fit_transform(df_input[cat_cols_train].astype(str))

In [5]:
df_input['engine_transmission'] = df_input['engine'] * df_input['transmission']

df_input['int_ext_color'] = df_input['int_col']*df_input['ext_col']

df_input.drop(columns = ['engine', 'transmission', 'int_col', 'ext_col'], inplace = True)

In [6]:
df_train_no_outliers = remove_outliers_iqr(df_input, 'milage')
df_train_no_outliers = remove_outliers_iqr(df_train_no_outliers, 'price')
df_train_no_outliers.reset_index(drop=True, inplace=True)
df = df_train_no_outliers

### 4. Preparação para Modelagem

In [7]:
seed = 42
y = df['price'] 
X = df.drop(['price'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

In [8]:
scaler = StandardScaler()
X_train.loc[:, ['brand','model', 'milage', 'int_ext_color', 'engine_transmission']] = scaler.fit_transform(X_train.loc[:, ['brand','model', 'milage', 'int_ext_color', 'engine_transmission']] )
X_test.loc[:, ['brand','model', 'milage', 'int_ext_color', 'engine_transmission']]  = scaler.transform(X_test.loc[:, ['brand','model', 'milage', 'int_ext_color', 'engine_transmission']] )

### 5. Treinamento e Avaliação do Modelo

In [9]:
# Testando diferentes números de vizinhos
for n_neighbors in np.arange(10,15):
    knn = KNeighborsRegressor(n_neighbors=n_neighbors)
    knn.fit(X_train, y_train)
    knn_predict = knn.predict(X_test)
    rmse = mean_squared_error(knn_predict, y_test, squared=False)
    print(f'RMSE KNN com {n_neighbors} vizinhos: {rmse:.2f}')

RMSE KNN com 10 vizinhos: 13920.11
RMSE KNN com 11 vizinhos: 13874.24
RMSE KNN com 12 vizinhos: 13831.95
RMSE KNN com 13 vizinhos: 13797.49
RMSE KNN com 14 vizinhos: 13765.18
